### Performance for updated training set 07/29/2025 ###
We have updates on data sets and prompts.
Use this code as a template to run predictions for the trainin and test sets

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import time
import logging
from pathlib import Path
from pandas.core.frame import DataFrame

logger = logging.getLogger(__name__)

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
from llmt.openaimodel import OpenAIModel
from llmt.performance import Performance

In [36]:
data_dir = os.path.join(os.environ.get('DATA'), 'hcp', 'hcp_update_250729')
label_file_name = 'outpatient_testing_set_5-29-2025.xlsx'
performance_file_name = 'outpatient_testing_set_5-29-2025-performance.parquet'
sample_file_name = 'outpatient_testing_set_5-29-2025-samples.parquet'

# results directory
output_dir = os.path.join(data_dir, 'output')
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Load the original data frame and save alphabetically by company name
df_label = pd.read_excel(os.path.join(data_dir, label_file_name))
df_label = df_label.\
    sort_values(by='Companies', ascending=True, key=lambda s: s.replace(' ','').str.lower()).\
    reset_index(drop=True)
output_label_file_name = 'outpatient_testing_set_5-29-2025.csv'
df_label.to_csv(os.path.join(output_dir, output_label_file_name), index=False)
display(df_label.head(2))

,Company ID,Companies,Description,Mental Health,Inpatient,Outpatient,Inpatient (including post-actute/residential)
0,525854-35,Vista Mental Health,Operator of a mental health clinic intended to...,1,0,1,0
1,503167-78,Vita Health,Operator of an online mental health care platf...,1,0,1,0


### Function to create the performance table ###

In [34]:
def performance_table(data: DataFrame, true_pred_cols: dict = None) -> DataFrame:
    if true_pred_cols is None:
        true_pred_cols = {'mental_health': 'pred_mh',
                          'inpatient': 'pred_ip',
                          'outpatient': 'pred_op'}
    performance_df_list = []
    for v, (true_col, pred_col) in enumerate(true_pred_cols.items()):
        performance_dict = Performance(data=data).binary_performance(true_col=true_col, pred_col=pred_col)
        performance_df = pd.DataFrame(performance_dict, index=[v])
        performance_df.insert(loc=0, column='category', value=true_col)
        performance_df_list.append(performance_df)
    performance_df = pd.concat(performance_df_list, axis=0)
    return performance_df

### Run models on Training data with evaluations ###

In [37]:
# Pick model and results 
model = 'gpt-4.1'
prompt = 2

col_dict = {'mental_health': 'pred_mh',
            'inpatient': 'pred_ip',
            'outpatient': 'pred_op',
            'mental_health_inpatient': 'pred_mh_ip',
            'mental_health_outpatient': 'pred_mh_op'}

df = pd.read_parquet(os.path.join(data_dir, sample_file_name))
df_model = df.loc[(df['model'] == model) & (df['prompt'] == prompt)]
df_model = df_model.\
    sort_values(by='name', ascending=True, key=lambda s: s.replace(' ','').str.lower()).\
    reset_index(drop=True)
df_perf = performance_table(data=df_model, true_pred_cols=col_dict)

output_sample_file_name = 'outpatient_testing_set_5-29-2025-gpt41.csv'
output_perf_file_name = 'outpatient_testing_set_5-29-2025-perf.csv'

df_model.to_csv(os.path.join(output_dir, output_sample_file_name), index=False)
df_perf.to_csv(os.path.join(output_dir, output_perf_file_name), index=False)

display(df_model.head(2))
display(df_perf)

,model,prompt,id,name,description,mental_health,inpatient,outpatient,Inpatient (including post-actute/residential),pred_mh,pred_ip,pred_op,verified_op,mental_health_inpatient,pred_mh_ip,mental_health_outpatient,pred_mh_op
0,gpt-4.1,2,525854-35,Vista Mental Health,Operator of a mental health clinic intended to...,1,0,1,0,1,0,1,1,0,0,1,1
1,gpt-4.1,2,503167-78,Vita Health,Operator of an online mental health care platf...,1,0,1,0,1,0,1,1,0,0,1,1


,category,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mental_health,162,38,125,31,7,37,0.7716,0.9470,0.810,0.8158,0.8503
1,inpatient,8,192,7,157,35,1,0.8750,0.1667,0.040,0.8177,0.2800
2,outpatient,154,46,129,39,7,25,0.8377,0.9485,0.770,0.8478,0.8897
3,mental_health_inpatient,7,193,6,160,33,1,0.8571,0.1538,0.035,0.8290,0.2609
4,mental_health_outpatient,132,68,108,57,11,24,0.8182,0.9076,0.660,0.8382,0.8606


In [41]:
# Create stats for the other inpatient column
df2 = copy.deepcopy(df_model)
df2 = df2.drop(['inpatient', 'mental_health_inpatient', 'pred_mh_ip'], axis=1).\
                rename(columns={'Inpatient (including post-actute/residential)': 'inpatient2'})
df2 = df2.\
    sort_values(by='name', ascending=True, key=lambda s: s.replace(' ','').str.lower()).\
    reset_index(drop=True)

# Combine the new inpatient columns
df2 = Performance(data=df2).\
    combine_columns(true_col_list=['mental_health', 'inpatient2'], 
                    pred_col_list=['pred_mh', 'pred_ip']).\
    rename(columns={'pred_mh_pred_ip': 'pred_mh_ip'})

col_dict = {'mental_health': 'pred_mh',
            'inpatient2': 'pred_ip',
            'outpatient': 'pred_op',
            'mental_health_inpatient2': 'pred_mh_ip',
            'mental_health_outpatient': 'pred_mh_op'}

df2_perf = performance_table(data=df2, true_pred_cols=col_dict)

df2 = df2.rename(columns={'inpatient2': 'Inpatient2 (including post-actute/residential)'})

output_sample_file_name = 'outpatient_testing_set_5-29-2025_2-gpt41.csv'
output_perf_file_name = 'outpatient_testing_set_5-29-2025_2-perf.csv'

df2.to_csv(os.path.join(output_dir, output_sample_file_name), index=False)
df2_perf.to_csv(os.path.join(output_dir, output_perf_file_name), index=False)

display(df2.head(2))
display(df2_perf)

,model,prompt,id,name,description,mental_health,outpatient,Inpatient2 (including post-actute/residential),pred_mh,pred_ip,pred_op,verified_op,mental_health_outpatient,pred_mh_op,mental_health_inpatient2,pred_mh_ip
0,gpt-4.1,2,525854-35,Vista Mental Health,Operator of a mental health clinic intended to...,1,1,0,1,0,1,1,1,1,0,0
1,gpt-4.1,2,503167-78,Vita Health,Operator of an online mental health care platf...,1,1,0,1,0,1,1,1,1,0,0


,category,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,mental_health,162,38,125,31,7,37,0.7716,0.9470,0.81,0.8158,0.8503
1,inpatient2,48,152,37,147,5,11,0.7708,0.8810,0.24,0.9671,0.8222
2,outpatient,154,46,129,39,7,25,0.8377,0.9485,0.77,0.8478,0.8897
3,mental_health_inpatient2,40,160,30,151,9,10,0.7500,0.7692,0.20,0.9437,0.7595
4,mental_health_outpatient,132,68,108,57,11,24,0.8182,0.9076,0.66,0.8382,0.8606
